## Stock Sentiment Analysis using News Headlines

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('Data.csv', encoding = "ISO-8859-1")

In [3]:
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,2000-01-05,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,...,South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,2000-01-06,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,...,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,2000-01-07,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,...,Most everywhere: UDIs,Most wanted: Chloe lunettes,Return of the cane 'completely off the agenda',From Sleepy Hollow to Greeneland,Blunkett outlines vision for over 11s,"Embattled Dobson attacks 'play now, pay later'...",Doom and the Dome,What is the north-south divide?,Aitken released from jail,Gone aloft


In [4]:
train = df[df['Date'] < '20150101']
test = df[df['Date'] > '20141231']

In [5]:
# Removing punctuations
data=train.iloc[:,2:3]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Renaming column names for ease of access
list1= [i for i in range(1)]
new_Index=[str(i) for i in list1]
data.columns= new_Index
data.head(5)



,0
0,A hindrance to operations extracts from the...
1,Scorecard
2,Coventry caught on counter by Flo
3,Pilgrim knows how to progress
4,Hitches and Horlocks


In [6]:
# Convertng headlines to lower case
for index in new_Index:
    data[index]=data[index].str.lower()
data.head(1)

,0
0,a hindrance to operations extracts from the...


In [7]:
' '.join(str(x) for x in data.iloc[1,0:1])

'scorecard'

In [8]:
headlines = []
for row in range(0,len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row,0:1]))

In [9]:
headlines[0]

'a  hindrance to operations   extracts from the leaked reports'

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [11]:
## implement BAG OF WORDS
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(headlines) 

In [12]:
# implement RandomForest Classifier
model=RandomForestClassifier(n_estimators=200,criterion='entropy')
model.fit(traindataset,train['Label'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
## Predict for the Test Dataset
test_transform= []
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:3]))
test_dataset = countvector.transform(test_transform)
predictions = model.predict(test_dataset)

In [14]:
## Import library to check accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [15]:
matrix=confusion_matrix(test['Label'],predictions)
print(matrix)
score=accuracy_score(test['Label'],predictions)
print(score)
report=classification_report(test['Label'],predictions)
print(report)

[[156  30]
 [ 30 162]]
0.8412698412698413
              precision    recall  f1-score   support

           0       0.84      0.84      0.84       186
           1       0.84      0.84      0.84       192

    accuracy                           0.84       378
   macro avg       0.84      0.84      0.84       378
weighted avg       0.84      0.84      0.84       378



In [16]:
def predict(string):
#     return model.predict(countvector.transform([string]))[0]
    return model.predict(countvector.transform([string]))[0]
    
predict(headlines[0])
# result=predict(headlines[5])
# result=predict("There is a 50% chance something bad will happen to the stock market soon")
# result=predict("Comcast Stock Gains On Upbeat 2020 View While Verizon And AT&T Slip")
# result=predict("AT&T CFO Addresses Stockholders, Expects Growth to Continue")
# print(result)
if(result):
    print("stock prize will go down")
else:
    print("stock prize will go up")
 #predict(headlines[7])

NameError: name 'result' is not defined

In [ ]:
headline=headlines[3]
print(headline)

In [ ]:
predictions

In [ ]:
import pickle
pickle.dump(model,open('model.pkl','wb'))

In [ ]:
# import pandas as pd
# from flask import Flask, jsonify, request
# import pickle
# import os

# # load model
# model = pickle.load(open(r'C:\Users\rohit\Documents\Stock Market Prediction\model.pkl','rb'))

# # app
# app = Flask(__name__)

# # routes
# @app.route('/', methods=['POST'])

# def predict():
#     # get data
#     data = request.get_json(force=True)

#     # convert data into dataframe
#     data.update((x, [y]) for x, y in data.items())
#     data_df = pd.DataFrame.from_dict(data)

#     # predictions
#     result = model.predict(data_df)

#     # send back to browser
#     output = {'results': int(result[0])}

#     # return data
#     return jsonify(results=output)

# if __name__ == '__main__':
#     app.run(port = 5000, debug=True)



from flask import Flask, request, redirect, url_for, flash, jsonify
import numpy as np
import pickle as p
import json


app = Flask(__name__)
# app = Flask(__name__) = pickle.load(open('model.pkl','rb'))


@app.route('/api/', methods=['POST'])
def makecalc():
    data = request.get_json()
    prediction = np.array2string(model.predict(data))

    return jsonify(prediction)

if __name__ == '__main__':
    modelfile = 'model.pkl'
    model = p.load(open(modelfile, 'rb'))
    app.run(debug=True, host='0.0.0.0')

In [ ]:
import json, requests, datetime
from flask import Flask, request, abort, jsonify
url = 'http://127.0.0.1:5000/predict'
j_data = json.dumps({"string":"There is a 50% chance something bad will happen to the stock market soon"})
# r = requests.post(url, data={"string":"There is a 50% chance something bad will happen to the stock market soon"})
r = requests.post(url, data=j_data)

print(r, r.text)
print(r.json())


In [ ]:
test_dataset.toarray()

In [17]:
print(train['Label'])

0       0
1       0
2       0
3       1
4       1
       ..
3970    0
3971    0
3972    1
3973    0
3974    0
Name: Label, Length: 3975, dtype: int64
